# Vertical and Temporal Profiles in emiproc

This tutorial shows basic handling of vertical and temporal profiles in emiproc.

- loading profiles from a file
- plotting profiles
- saving profiles to a file

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:

import numpy as np

import matplotlib.pyplot as plt

import emiproc
from emiproc.profiles.temporal_profiles import (
    TemporalProfile,
    DailyProfile,
    WeeklyProfile,
    MounthsProfile,
    create_scaling_factors_time_serie,
    from_csv,
    from_yaml,
    to_yaml,
    read_temporal_profiles,
)
from emiproc.profiles import vertical_profiles 
from emiproc.tests_utils.temporal_profiles import copernicus_profiles_dir

## Vertical profiles

In [ ]:
# Load from csv

profiles, cat_sub = vertical_profiles.from_csv(
    copernicus_profiles_dir / "vertical.csv"
)   

In [ ]:
# Plot the profiles
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
for name, profile in zip(cat_sub, profiles.ratios):
    ax.plot(profile, label=name)
ax.legend()

## Time profiles

In [ ]:
read_temporal_profiles(copernicus_profiles_dir, profile_csv_kwargs={"sep": ","})

In [ ]:
# Profiles from a csv profiles file 


profiles = ['hour_in_day', 'day_in_week', 'month_in_year']
profiles = {p: from_csv(copernicus_profiles_dir / f"timeprofiles-{p}.csv") for p in profiles}

# Plot the profiles on 3 subplots and put the legend on a 4rth 
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs = axs.flatten()
for ax, (name, profile) in zip(axs, profiles.items()):
    for cat_name, cat_profile in profile.items():
        ax.plot(cat_profile.ratios, label=cat_name)
    ax.set_title(name)

for cat_name, cat_profile in profile.items():
    axs[-1].plot([], label=cat_name)
axs[-1].legend()

In [ ]:
# Profiles from a yaml profiles file
yaml_dir = emiproc.FILES_DIR / "profiles" / "yamls"
yaml_profiles = {}
for yml_file in yaml_dir.glob("*.yaml"):
    yaml_profiles[yml_file.stem] = from_yaml(yml_file)

# Now with the yaml, each catoegory can have different types of profiles
yaml_profiles['heat'], yaml_profiles['heavy']


In [ ]:
# We can create an hourly time series of each categories to plot
tss = {}
for categorie in yaml_profiles.keys():
    tss[categorie] = create_scaling_factors_time_serie(
        start_time="2020-01-01",
        end_time="2022-04-28",
        profiles=yaml_profiles[categorie],
    )
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# Create better colors 
colors = iter(plt.cm.tab10(np.linspace(0, 1, len(tss))))
for name, ts in tss.items():
    ax.plot(ts, label=name, color=next(colors))
ax.legend()

In [ ]:
# We can also save time profiles to yaml
for categorie in yaml_profiles.keys():
    yaml_file = emiproc.FILES_DIR / 'outputs' / f'test_{categorie}.yaml'
    to_yaml(yaml_profiles[categorie], yaml_file)


In [ ]:
# Profiles country dependent from an other data source:


p, i = read_temporal_profiles(
    # Change the path properly
    "emission-time-profiles/time_profiles/emiproc",
    profile_csv_kwargs={"sep": ","},
    rtol=1e-2,
)
i

### Profiles example in Invenotries (TNO example)

In [ ]:
from pathlib import Path
import xarray as xr

from emiproc.inventories.tno import TNO_Inventory

tno_nc_file = Path(r"C:\Users\coli\Documents\emiproc\files\TNO_6x6_GHGco_v4_0\TNO_GHGco_v4_0_year2018.nc")


inv_tno = TNO_Inventory(tno_nc_file)

In [ ]:
inv_tno.t_profiles_groups[-1]

In [ ]:
# Add a profile to a category of the inventory
inv_tno.set_profile(yaml_profiles['heat'], category='J')

### ICON-OEM Profiles

the following example shows how to load profiles created for ICON - OEM

In [ ]:
from pathlib import Path
import xarray as xr

nc_file = Path(r"C:\Users\coli\Documents\ZH-CH-emission\icon_europe_DOM01_with_tno_emissions\hourofyear.nc")

ds = xr.open_dataset(nc_file)
ds


In [ ]:
%matplotlib widget
plt.figure()
cat = "GNFR_A-CO2"
country = 2
ds[cat].sel(country=country).plot()
plt.show()